# Preços de imóveis em João Pessoa

## Introdução

Esta análise se baseia na raspagem de dados na OLX para a pesquisa <a href = "https://www.olx.com.br/brasil?q=im%C3%B3veis%20em%20jo%C3%A3o%20pessoa&o=1">imóveis em joão pessoa</a>, seguida de análise exploratória desses dados.

São empregadas para tanto as seguintes bibliotecas:

- Requests para a realização de requisições HTTP;

- time para intervalos entre requisições;

- random para randomizar intervalos(otimização);

- tqdm para controle de tempo;

- BeautifulSoup para a extração de dados do html das páginas;

- re para a filtragem mais fina desses dados;

- Pandas para manipulação de dados em dataframes.

## 1. Importar bibliotecas

In [1]:
import requests
import time
from random import uniform
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv

## 2. Web scraping

A primeira parte de nossa análise será a extração de dados por scraping do site da OLX. Para tanto procederemos assim:

1. Inicialmente varremos as páginas de pesquisa à procura dos links de cada um dos produtos;

2. Depois, para cada produto, faremos a raspagem de suas informações em sua própria página.

In [2]:
base_url = 'https://www.olx.com.br/brasil?q=im%C3%B3veis%20em%20jo%C3%A3o%20pessoa'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

products_links = []
max_consecutive_empty_pages = 3
consecutive_empty_pages = 0
page = 0

# Criando o tqdm apenas para mostrar o tempo estimado
pbar = tqdm(total=10, desc="Processando", unit="página", bar_format='{desc}: {n_fmt}/{total_fmt} [Tempo restante: {remaining}]')

while True:
    page += 1
    
    url = f"{base_url}&o={page}"

    try:
        search_response = requests.get(url, headers=headers)

        if search_response.status_code == 200:
            search_soup = BeautifulSoup(search_response.text, 'html.parser')

            no_results_element = search_soup.find('div', class_='empty-list')
            if no_results_element or 'Nenhum resultado encontrado' in search_response.text:
                pbar.set_postfix_str("Não há mais resultados disponíveis")
                break

            all_links = search_soup.find_all('a', class_='olx-adcard__link')
            
            if all_links:
                consecutive_empty_pages = 0
                for link in all_links:
                    href = link.get('href')
                    if href and href.startswith('http'):
                        products_links.append(href)
            else:
                consecutive_empty_pages += 1
                pbar.set_postfix_str(f"Pág {page} vazia ({consecutive_empty_pages} consecutivas)")
                
                if consecutive_empty_pages >= max_consecutive_empty_pages:
                    pbar.set_postfix_str(f"Parando após {max_consecutive_empty_pages} páginas vazias")
                    break

        elif search_response.status_code == 404:
            pbar.set_postfix_str("Página não encontrada (404)")
            break
        else:
            pbar.set_postfix_str(f"Erro: {search_response.status_code}")
            
    except Exception as e:
        pbar.set_postfix_str(f"Erro: {e}")
        
    if page >= 10:
        pbar.set_postfix_str("Limite de 10 páginas atingido")
        break

    # Atualizar o progresso para calcular o tempo restante
    pbar.update(1)
    pbar.set_postfix_str(f"Produtos: {len(products_links)}")
    
    time.sleep(uniform(1, 3))

pbar.close()
print(f"Processamento concluído! Total de produtos encontrados: {len(products_links)}")

Processando: 0/10 [Tempo restante: ?]

Processamento concluído! Total de produtos encontrados: 500


Vejamos os dez primeiros links que coletamos:

In [3]:
products_links[0:10]

['https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-em-joao-pessoa-1427390473',
 'https://pb.olx.com.br/paraiba/imoveis/vendo-imoveis-de-alto-padrao-a-beira-mar-de-intermares-1399963055',
 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-1424880641',
 'https://pb.olx.com.br/paraiba/imoveis/imoveis-exclusivos-1422666330',
 'https://pb.olx.com.br/paraiba/servicos/imoveis-vende-ou-aluga-casa-apartamento-terreno-1348330489',
 'https://pb.olx.com.br/paraiba/imoveis/alugo-imoveis-1418639984',
 'https://pb.olx.com.br/paraiba/servicos/reforma-de-imoveis-em-joao-pessoa-pisos-pintura-e-outros-acabamentos-1424786399',
 'https://pb.olx.com.br/paraiba/servicos/corretor-de-imoveis-em-joao-pessoa-1357257740',
 'https://pb.olx.com.br/paraiba/servicos/seu-corretor-de-imoveis-perito-avaliador-em-joao-pessoa-1385905983',
 'https://pb.olx.com.br/paraiba/imoveis/apartamentos-para-venda-varios-imoveis-1425816782']

São exatamente os dez links que aparecem na primeira página de busca! Vamos agora investigá-los individualmente:

In [4]:
all_products = []

# Criando a barra de progresso tqdm
pbar = tqdm(total=len(products_links), desc="Processando produtos", unit="produto", bar_format='{desc}: {n_fmt}/{total_fmt} [Tempo restante: {remaining}]')

for i, link in enumerate(products_links, 1):
    try:
        product_response = requests.get(link, headers=headers, timeout=10)

        if product_response.status_code == 200:
            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            title_elem = product_soup.find('span', class_='olx-text olx-text--title-medium olx-text--block ad__sc-1l883pa-2 bdcWAn')
            title = title_elem.text.strip() if title_elem else 'Não especificado'

            location_div = product_soup.find('div', id='location')
            if location_div:
                location_elem = location_div.find('span', class_='olx-text olx-text--body-small olx-text--block olx-text--semibold olx-color-neutral-110')
                location = location_elem.text.strip() if location_elem else 'Não especificado'
            else:
                location = 'Não especificado'
            
            price_div = product_soup.find('div', id='price-box-container')
            if price_div:
                price_elem = price_div.find('span', {'data-ds-component': 'DS-Text'})
                price = price_elem.text.strip() if price_elem else 'Não especificado'
                
                product_type_elem = price_div.find('span', class_='olx-badge olx-badge--neutral olx-badge--small olx-badge--rectangle')
                product_type = product_type_elem.text.strip() if product_type_elem else "Não especificado"
            else:
                price = 'Não especificado'
                product_type = "Não especificado"

            product_data = {
                'title': title,
                'location': location,
                'price': price,
                'product_type': product_type,
                'link': link  
            }

            all_products.append(product_data)
            
            # Atualizar informações na barra de progresso
            pbar.set_postfix_str(f"Último: {title[:20]}...")

        else:
            pbar.set_postfix_str(f"Erro: Status {product_response.status_code}")

    except Exception as e:
        pbar.set_postfix_str(f"Erro: {str(e)[:20]}...")
    
    # Atualizar o progresso
    pbar.update(1)
    time.sleep(uniform(1, 3))

pbar.close()
print(f"Extração concluída! Total de produtos: {len(all_products)}")

Processando produtos: 0/500 [Tempo restante: ?]

Extração concluída! Total de produtos: 495


In [5]:
all_products[0:10]

[{'title': 'Imóveis na planta em João Pessoa',
  'location': 'Cabo Branco, João Pessoa, PB, 58045010',
  'price': 'R$ 274.900',
  'product_type': 'Venda',
  'link': 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-em-joao-pessoa-1427390473'},
 {'title': 'Vendo Imóveis de Alto padrão a Beira Mar de Intermares',
  'location': 'Ipês, João Pessoa, PB, 58028530',
  'price': 'R$ 480.000',
  'product_type': 'Venda',
  'link': 'https://pb.olx.com.br/paraiba/imoveis/vendo-imoveis-de-alto-padrao-a-beira-mar-de-intermares-1399963055'},
 {'title': 'Imóveis na Planta',
  'location': 'Jardim Oceania, João Pessoa, PB, 58037030',
  'price': 'R$ 416.234',
  'product_type': 'Venda',
  'link': 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-1424880641'},
 {'title': 'Imoveis exclusivos',
  'location': 'Manaíra, João Pessoa, PB, 58038142',
  'price': 'R$ 650.000',
  'product_type': 'Venda',
  'link': 'https://pb.olx.com.br/paraiba/imoveis/imoveis-exclusivos-1422666330'},
 {'title': 'IMOVEI

Para o restante da análise, incluindo a limpeza e o tratamento dos dados, passamos as informações extraídas para um dataframe, de modo a empregar os artifícios da biblioteca Pandas.

## 3. Visualização inicial

In [131]:
df = pd.DataFrame(all_products)

In [132]:
df

,title,location,price,product_type,link
0,Imóveis na planta em João Pessoa,"Cabo Branco, João Pessoa, PB, 58045010",R$ 274.900,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,"Ipês, João Pessoa, PB, 58028530",R$ 480.000,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,"Jardim Oceania, João Pessoa, PB, 58037030",R$ 416.234,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,"Manaíra, João Pessoa, PB, 58038142",R$ 650.000,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
4,IMOVEIS VENDE OU ALUGA CASA APARTAMENTO TERRENO,"João Pessoa, PB, 58052030",Não especificado,Não especificado,https://pb.olx.com.br/paraiba/servicos/imoveis...
...,...,...,...,...,...
490,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066227",R$ 66.495,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
491,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58068343",R$ 92.444,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
492,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066217",R$ 82.813,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
493,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58065033",R$ 87.819,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         495 non-null    object
 1   location      495 non-null    object
 2   price         495 non-null    object
 3   product_type  495 non-null    object
 4   link          495 non-null    object
dtypes: object(5)
memory usage: 19.5+ KB


Algumas pessoas usaram indevidamente a OLX para disponibilizar seus contatos, a fim de realizar serviços por fora do aplicativo. Podemos remover as linhas correspondentes a esses erros usando verificação por expressão regular na linha de preços, e aproveitar, inclusive, para corrigir a apresentação dos preços para um valor numérico:

In [134]:
# Filtrar para regex correto
price_regex = r'R\$\s*\d{1,3}(?:\.\d{3})*(?:,\d{1,2})?|R\$\s*\d+(?:,\d{1,2})?'
price_mask = df['price'].apply(lambda x: bool(re.search(price_regex, str(x))))
df = df[price_mask]

In [135]:
df

,title,location,price,product_type,link
0,Imóveis na planta em João Pessoa,"Cabo Branco, João Pessoa, PB, 58045010",R$ 274.900,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,"Ipês, João Pessoa, PB, 58028530",R$ 480.000,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,"Jardim Oceania, João Pessoa, PB, 58037030",R$ 416.234,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,"Manaíra, João Pessoa, PB, 58038142",R$ 650.000,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
5,ALUGO IMÓVEIS,"Treze de Maio, João Pessoa, PB, 58025430",R$ 800,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
490,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066227",R$ 66.495,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
491,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58068343",R$ 92.444,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
492,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066217",R$ 82.813,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
493,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58065033",R$ 87.819,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


In [136]:
df.loc[:, 'price'] = (
    df['price']
    .str.replace('R\$\s*', '', regex=True)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
    .astype('float64')
)

In [137]:
df

,title,location,price,product_type,link
0,Imóveis na planta em João Pessoa,"Cabo Branco, João Pessoa, PB, 58045010",274900.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,"Ipês, João Pessoa, PB, 58028530",480000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,"Jardim Oceania, João Pessoa, PB, 58037030",416234.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,"Manaíra, João Pessoa, PB, 58038142",650000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
5,ALUGO IMÓVEIS,"Treze de Maio, João Pessoa, PB, 58025430",800.0,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
490,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066227",66495.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
491,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58068343",92444.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
492,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066217",82813.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
493,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58065033",87819.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


Outra coisa que faremos é transformar a coluna ``location`` em uma coluna contendo apenas os CEPs de cada imóvel (já aproveitamos para deixar o CEP correto para a cidade de João Pessoa):

In [138]:
df = df.rename(columns = {'location': 'cep'})

In [139]:
df

,title,cep,price,product_type,link
0,Imóveis na planta em João Pessoa,"Cabo Branco, João Pessoa, PB, 58045010",274900.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,"Ipês, João Pessoa, PB, 58028530",480000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,"Jardim Oceania, João Pessoa, PB, 58037030",416234.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,"Manaíra, João Pessoa, PB, 58038142",650000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
5,ALUGO IMÓVEIS,"Treze de Maio, João Pessoa, PB, 58025430",800.0,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
490,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066227",66495.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
491,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58068343",92444.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
492,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066217",82813.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
493,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58065033",87819.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


In [140]:
cep_regex = r'(58\d{6})'

df.loc[:, 'cep'] = df['cep'].str.extract(cep_regex)
df = df.dropna(subset=['cep']).copy()
df.reset_index(drop = True)

,title,cep,price,product_type,link
0,Imóveis na planta em João Pessoa,58045010,274900.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,58028530,480000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,58037030,416234.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,58038142,650000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
4,ALUGO IMÓVEIS,58025430,800.0,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
463,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,58066227,66495.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
464,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,58068343,92444.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
465,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,58066217,82813.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
466,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,58065033,87819.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


Em ``title``, convém remover vírgulas e pontuações, bem como normalizar o texto para lower case:

In [120]:
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['title'] = df['title'].apply(lambda x: x.lower())

In [121]:
df

,title,cep,price,product_type,link
0,imóveis na planta em joão pessoa,58045010,274900.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,vendo imóveis de alto padrão a beira mar de in...,58028530,480000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,imóveis na planta,58037030,416234.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,imoveis exclusivos,58038142,650000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
5,alugo imóveis,58025430,800.0,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
490,oportunidade única em joao pessoa pb tipo ap...,58066227,66495.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
491,oportunidade única em joao pessoa pb tipo ap...,58068343,92444.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
492,oportunidade única em joao pessoa pb tipo ap...,58066217,82813.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
493,oportunidade única em joao pessoa pb tipo ap...,58065033,87819.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


In [141]:
df.to_csv('data.csv')